In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("~/Desktop/videos/iOS/test2/loss10_1.csv")
df['PIX_VALUE_DIFF'] = df['TOTAL_PIX_VALUE'].diff()
df.loc[(df['PIX_VALUE_DIFF'] <= 100) & (df['PIX_VALUE_DIFF'] >= -100), 'PIX_VALUE_DIFF'] = 0
df = df.fillna(0)
print(df.head(60))

        TIME  TOTAL_PIX_VALUE  PIX_VALUE_DIFF
0   0.019658            20767             0.0
1   0.039317            20767             0.0
2   0.058975            16304         -4463.0
3   0.078633            16294             0.0
4   0.098291            16297             0.0
5   0.117950            16298             0.0
6   0.137608            16302             0.0
7   0.157266            16298             0.0
8   0.176924            16302             0.0
9   0.196583            16298             0.0
10  0.216241            16302             0.0
11  0.235899            16298             0.0
12  0.255557            16302             0.0
13  0.275216            13218         -3084.0
14  0.294874            13219             0.0
15  0.314532            13215             0.0
16  0.334190            13213             0.0
17  0.353849            13213             0.0
18  0.373507            13210             0.0
19  0.393165            13215             0.0
20  0.412824            13215     

In [3]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [4]:
time_period = zero_runs(df['PIX_VALUE_DIFF'].tolist())-1
time_period[0, :1] = 0
print(time_period)
print("Number of PIX_VALUE_DIFF which value is 0: %s"%len(time_period))

[[   0    1]
 [   2   12]
 [  13   24]
 [  25   34]
 [  35   44]
 [  45   55]
 [  56   65]
 [  66   75]
 [  76   86]
 [  87   95]
 [  96  104]
 [ 105  116]
 [ 117  125]
 [ 126  136]
 [ 137  145]
 [ 146  157]
 [ 158  166]
 [ 167  177]
 [ 178  187]
 [ 188  196]
 [ 197  205]
 [ 206  216]
 [ 217  227]
 [ 228  237]
 [ 238  247]
 [ 248  258]
 [ 259  268]
 [ 269  279]
 [ 280  289]
 [ 290  297]
 [ 298  307]
 [ 308  318]
 [ 319  329]
 [ 330  338]
 [ 339  350]
 [ 351  359]
 [ 360  370]
 [ 371  381]
 [ 382  391]
 [ 392  400]
 [ 401  410]
 [ 411  419]
 [ 420  430]
 [ 431  440]
 [ 441  451]
 [ 452  462]
 [ 463  472]
 [ 473  483]
 [ 484  493]
 [ 494  501]
 [ 502  512]
 [ 513  522]
 [ 523  533]
 [ 534  543]
 [ 544  554]
 [ 555  564]
 [ 565  575]
 [ 576  586]
 [ 587  595]
 [ 596  605]
 [ 606  614]
 [ 615  625]
 [ 626  636]
 [ 637  644]
 [ 645  656]
 [ 657  666]
 [ 667  677]
 [ 678  688]
 [ 689  698]
 [ 699  707]
 [ 708  717]
 [ 718  726]
 [ 727  738]
 [ 739  747]
 [ 748  758]
 [ 759  769]
 [ 770  777]

In [5]:
THRESHOLD = 11 # lag for more than 7 frames
filtered_time_period = time_period[np.where(np.diff(time_period)+1 >= THRESHOLD)[0]]

In [6]:
result = pd.DataFrame(columns=['START_TIME', 'END_TIME', 'TIME_PERIOD'])

for i in range(0, len(filtered_time_period)):
    start_time = df.iloc[filtered_time_period[i][0]]['TIME']
    end_time = df.iloc[filtered_time_period[i][1]]['TIME']
    time_diff = end_time - start_time
    result = result.append({'START_TIME': start_time, 'END_TIME': end_time, 'TIME_PERIOD': time_diff}, ignore_index=True)

In [7]:
print(result)

     START_TIME   END_TIME  TIME_PERIOD
0      0.058975   0.255557     0.196583
1      0.275216   0.491457     0.216241
2      0.904280   1.100863     0.196583
3      1.513686   1.710269     0.196583
4      2.083776   2.300017     0.216241
5      2.496599   2.693182     0.196583
6      2.889765   3.106006     0.216241
7      3.302588   3.499171     0.196583
8      4.069261   4.265843     0.196583
9      4.285501   4.482084     0.196583
10     4.894908   5.091490     0.196583
11     5.307731   5.504314     0.196583
12     6.074403   6.270986     0.196583
13     6.290644   6.487227     0.196583
14     6.683810   6.900050     0.216241
15     7.096633   7.293216     0.196583
16     7.312874   7.509457     0.196583
17     8.276129   8.472712     0.196583
18     8.688952   8.885535     0.196583
19     8.905193   9.101776     0.196583
20     9.318017   9.514599     0.196583
21     9.888106  10.084689     0.196583
22    10.300930  10.497513     0.196583
23    10.713754  10.910336     0.196583


In [8]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [9]:
p = figure(width=500, height=500)
p.circle(result['START_TIME'], result['TIME_PERIOD'], size=5, color="firebrick", alpha=0.5)
show(p)